In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import Token_NOAA
from config import API_Token
import calendar
#Suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
#Display all columns
pd.set_option('display.max_columns', None)

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

# Weather Data By Zip Code

In [3]:
napa_df = pd.DataFrame()
states = {
    'Napa': {
        'zip': '94558',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Columbia': {
        'zip': '98813',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Sonoma': {
        'zip': '95476',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Santa': {
        'zip': '95062',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Yakima': {
        'zip': '98903',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
    'Dundee': {
        'zip': '97045',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [6]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
#         print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if napa_df.empty:
        napa_df = small_df
    else:
        napa_df = pd.merge(napa_df,small_df)
napa_df.to_csv( 'Napa_Temp_Prcp_Napa.csv', index = False)

working on year 1992
{'metadata': {'resultset': {'offset': 1, 'count': 1005, 'limit': 1000}}, 'results': [{'date': '1992-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '1992-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 55.0}, {'date': '1992-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 33.0}, {'date': '1992-01-02T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '1992-01-02T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 56.0}, {'date': '1992-01-02T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 33.0}, {'date': '1992-01-03T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '19

working on year 1993
{'metadata': {'resultset': {'offset': 1, 'count': 1095, 'limit': 1000}}, 'results': [{'date': '1993-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 1.23}, {'date': '1993-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 53.0}, {'date': '1993-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 44.0}, {'date': '1993-01-02T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.01}, {'date': '1993-01-02T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 51.0}, {'date': '1993-01-02T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 28.0}, {'date': '1993-01-03T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '19

working on year 1994
{'metadata': {'resultset': {'offset': 1, 'count': 1092, 'limit': 1000}}, 'results': [{'date': '1994-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.02}, {'date': '1994-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 52.0}, {'date': '1994-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 37.0}, {'date': '1994-01-02T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.01}, {'date': '1994-01-02T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 50.0}, {'date': '1994-01-02T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 41.0}, {'date': '1994-01-03T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '19

working on year 1995
{'metadata': {'resultset': {'offset': 1, 'count': 1005, 'limit': 1000}}, 'results': [{'date': '1995-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.21}, {'date': '1995-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 45.0}, {'date': '1995-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 38.0}, {'date': '1995-01-02T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.06}, {'date': '1995-01-02T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 48.0}, {'date': '1995-01-02T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 36.0}, {'date': '1995-01-03T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.83}, {'date': '19

working on year 1996
{'metadata': {'resultset': {'offset': 1, 'count': 1098, 'limit': 1000}}, 'results': [{'date': '1996-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '1996-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 71.0}, {'date': '1996-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 44.0}, {'date': '1996-01-02T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '1996-01-02T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 74.0}, {'date': '1996-01-02T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 47.0}, {'date': '1996-01-03T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': 'P,,0,0800', 'value': 0.0}, {'date': '19

...

working on year 2005
{'metadata': {'resultset': {'offset': 1, 'count': 3783, 'limit': 1000}}, 'results': [{'date': '2005-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.58}, {'date': '2005-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 54.0}, {'date': '2005-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 37.0}, {'date': '2005-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 0.55}, {'date': '2005-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 56.0}, {'date': '2005-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 36.0}, {'date': '2005-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USW00093227', 'attributes': ',,0,2400', 'value': 0.02}, {'date': '20

working on year 2006
{'metadata': {'resultset': {'offset': 1, 'count': 4200, 'limit': 1000}}, 'results': [{'date': '2006-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00044673', 'attributes': ',,0,1600', 'value': 0.63}, {'date': '2006-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00044673', 'attributes': ',I,0,1600', 'value': 52.0}, {'date': '2006-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00044673', 'attributes': ',,0,1600', 'value': 50.0}, {'date': '2006-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.02}, {'date': '2006-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 60.0}, {'date': '2006-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 43.0}, {'date': '2006-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00049859', 'attributes': 'P,,0,0730', 'value': 0.0}, {'date': '2

working on year 2007
{'metadata': {'resultset': {'offset': 1, 'count': 4072, 'limit': 1000}}, 'results': [{'date': '2007-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00044673', 'attributes': 'P,,0,1600', 'value': 0.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 0.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 55.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00045360', 'attributes': ',,0,0800', 'value': 33.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 0.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 69.0}, {'date': '2007-01-01T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00049859', 'attributes': ',,0,0730', 'value': 28.0}, {'date': '2007

working on year 2008


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
walladf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Walla': {
        'zip': '99362',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [7]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if walladf.empty:
        walladf = small_df
    else:
        walladf = pd.merge(walladf,small_df)
walladf.head(20)

working on year 1992
working on year 1993


KeyboardInterrupt: 

In [8]:
columbiadf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Columbia': {
        'zip': '98813',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [9]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if columbiadf.empty:
        columbiadf = small_df
    else:
        columbiadf = pd.merge(columbiadf,small_df)
columbiadf.to_csv( 'Columbia_Temp_Prcp.csv', index = False)

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008
working on year 2009


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
sonomadf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Sonoma': {
        'zip': '95476',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if sonomadf.empty:
        sonomadf = small_df
    else:
        sonomadf = pd.merge(sonomadf,small_df)
sonomadf.to_csv( 'Sonoma_Temp_Prcp.csv', index = False)

In [ ]:
santadf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Santa': {
        'zip': '95062',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if santadf.empty:
        santadf = small_df
    else:
        santadf = pd.merge(santadf,small_df)
santadf.to_csv( 'Santa_Temp_Prcp.csv', index = False)

In [ ]:
yakimadf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Yakima': {
        'zip': '98903',
        'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if yakimadf.empty:
        yakimadf = small_df
    else:
        yakimadf = pd.merge(yakimadf,small_df)
yakimadf.to_csv( 'Yakima_Temp_Prcp_Napa.csv', index = False)

In [ ]:
dundeedf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Dundee': {
        'zip': '97045',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        },
#     'Willamette': {
#         'zip': '00077',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         }
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if dundeedf.empty:
        dundeedf = small_df
    else:
        dundeedf = pd.merge(dundeedf,small_df)
dundeedf.to_csv( 'Dundee_Temp_Prcp_Napa.csv', index = False)

In [ ]:
willamettedf = pd.DataFrame()
states = {
#     'Napa': {
#         'zip': '94558',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Walla': {
#         'zip': '99362',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Columbia': {
#         'zip': '98813',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Sonoma': {
#         'zip': '95476',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Santa': {
#         'zip': '95062',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Yakima': {
#         'zip': '98903',
#         'years':['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
#     'Dundee': {
#         'zip': '97045',
#         'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
#         },
    'Willamette': {
        'zip': '00077',
        'years': ['1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015', '2016', '2017']
        }
}

In [ ]:
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #get value field for max, min, prcp
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
        items_PRCP = [item for item in d['results'] if item['datatype']=='PRCP']
        #get date field for max, min, prcp
        dates_temp_MAX = [item['date'].split('T')[0] for item in items_MAX]
        dates_temp_MIN = [item['date'].split('T')[0] for item in items_MIN]
        dates_temp_PRCP = [item['date'].split('T')[0] for item in items_PRCP]
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_MAX date/ loop through values
        df_test['date'] = dates_temp_MAX
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_Min date/ loop through values
        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
#------------------------------------------------------------------------------ Create col avgMaxTemp/ loop through items_PRCP date/ loop through values
        df_test['avgPRCP'+state]=np.nan
        for item in items_PRCP:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgPRCP'+state]] = item['value']
#------------------------------------------------------------------------------Combine the dataframes          
        small_df = pd.concat([small_df,df_test])
    if willamettedf.empty:
        willamettedf = small_df
    else:
        willamettedf = pd.merge(willamettedf,small_df)
willamettedf.to_csv( 'Williamette_Temp_Prcp.csv', index = False)

In [ ]:
result = pd.concat([napa_df, walladf, columbiadf, sonomadf, santadf, yakimadf, dundeedf, willamettedf], ignore_index=True, sort=False)


In [ ]:
bigdf.to_csv( 'daily_weather_Temp_Prcp_Napa.csv', index = False)
bigdf.date = pd.to_datetime(bigdf['date'])
bigdf=bigdf.set_index('date').resample('M').mean()

In [ ]:
bigdf

In [ ]:
#dataframe save as csv file
# bigdf.to_csv( 'daily_weather_Temp_Prcp_2017.csv', index = False)
# imported_df=pd.read_csv('daily_weather_Temp_Prcp_2017.csv')
# imported_df.head(50)

In [ ]:
bigdf.date = pd.to_datetime(bigdf['date'])
bigdf=bigdf.set_index('date').resample('M').mean()

In [ ]:
bigdf

# Split Date Column into months and year columns

In [ ]:
bigdf.reset_index(inplace= True )
bigdf['date']=pd.to_datetime(bigdf['date'])
bigdf.reset_index(inplace=True)
bigdf['day'] = bigdf['date'].dt.day
bigdf['month'] = bigdf['date'].dt.month
bigdf['year'] = bigdf['date'].dt.year

list_months=[]
for x in bigdf['month']:
    list_months.append(calendar.month_name[x])
bigdf['months']=list_months
bigdf.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
bigdf